In [2]:
#!pip install SPARQLWrapper

In [24]:
from SPARQLWrapper import SPARQLWrapper, JSON

entities = ["http://dbpedia.org/resource/Steve_Jobs", "http://dbpedia.org/resource/Apple_Inc."]

def send_query(query=""):
    sparql = SPARQLWrapper("http://dbpedia.org/sparql")
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    return results


def forward_query(entity):
    forward_query = f"""SELECT DISTINCT ?s ?o WHERE {{
 ?s ?p ?o .
  FILTER(?s = <{entity}>) .
  FILTER(!isLiteral(?o))
}}"""
    results = send_query(query=forward_query)
    #print(results)
    return { entity : {x['o']['value'] for x in results['results']['bindings']}}

    
def backward_query(entity):
    backward_query = f"""SELECT DISTINCT ?s ?o WHERE {{
 ?s ?p ?o .
  FILTER(?o = <{entity}>) .
  FILTER(!isLiteral(?s))
}}"""
    results = send_query(query=backward_query)
    return { entity : {x['o']['value'] for x in results['results']['bindings']}}

entity_dict = {}
for entity in entities:
    # ent -> x
    entity_dict_forward = forward_query(entity)
    # x -> ent
    entity_dict_backward = backward_query(entity)

    # For the ones we went forward towards
    for sub_entity in entity_dict_forward[entity]:
        print(sub_entity)
        # ent -> x . x -> x'
        entity_dict_forward_forward = forward_query(sub_entity)
        # ent -> x . x' -> x
        entity_dict_forward_backward = backward_query(sub_entity)
        
    # For the ones we went backward to
    for sub_entity in entity_dict_backward[entity]:
        print(sub_entity)
        # x -> ent . x -> x'
        entity_dict_backward_forward = forward_query(sub_entity)
        # x -> ent . x' -> x
        entity_dict_backward_backward = backward_query(sub_entity)
    
    entity_dict[entity] = { 'f': entity_dict_forward[entity], 
                           'b': entity_dict_backward[entity], 
                           'ff': entity_dict_forward_forward[entity],
                           'fw': entity_dict_forward_backward[entity],
                           'bw': entity_dict_backward_forward[entity],
                           'bb': entity_dict_backward_backward[entity]
                          }
    
    



http://dbpedia.org/resource/History_of_bottle_recycling_in_the_United_States
http://dbpedia.org/resource/Sun_Microsystems
http://dbpedia.org/resource/USB
http://dbpedia.org/class/yago/Specialist110631941
http://dbpedia.org/class/yago/Scholar110557854
http://dbpedia.org/resource/WALL-E
http://mk.dbpedia.org/resource/Стив_Џобс
http://kn.dbpedia.org/resource/ಸ್ಟೀವ್_ಜಾಬ್ಸ್
http://dbpedia.org/resource/Joanna_Hoffman
http://vec.dbpedia.org/resource/Steve_Jobs
http://dbpedia.org/class/yago/WikicatAmericanPeople
http://dbpedia.org/resource/Template:See_also
http://dbpedia.org/resource/Category:American_technology_chief_executives
http://commons.wikimedia.org/wiki/Special:FilePath/Steve_Jobs_Headshot_2010-CROP_(cropped_2).jpg
http://dbpedia.org/resource/Computer_mouse
http://dbpedia.org/resource/Printed_circuit_board
http://dbpedia.org/resource/Student_government_president
http://dbpedia.org/resource/File:SteveJobs_house_in_PaloAlto_with_fruit_trees.jpg
http://dbpedia.org/resource/File:Steve_Jo

AttributeError: 'set' object has no attribute 'values'